**Sample Forex Factory webscraper**

1. Install and import dependencies

In [1]:
pip install bs4

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install chromedriver-binary

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import chromedriver_binary

2. Create a browser instance

In [6]:
#Creating a webdriver instance
driver = webdriver.Edge()
#driver = webdriver.Chrome()         ... in case you have google chrome installed

3. Use browser instance to access web pages

In [7]:
# Opening website
driver.get("https://www.forexfactory.com")

4. Extract HTML from web page

In [8]:
#Extracting data from website
src = driver.page_source
# Now using beautiful soup
mysoup = BeautifulSoup(src, 'html')

5. Extract specific HTML element

In [9]:
#Getting the 'today' section
today = mysoup.find("tbody")

6. List all upcoming events

In [10]:
#If a news item has a red marker, we will add it to our array of important events.

all_events = today.find_all("tr", {"class": "calendar__row"})
#newsitems = username_element.get_text()

7. Filter out important events only

In [11]:
coming_soon = []  # List of all events that are coming soon
imp_events = []  # List of important events
high_imp = []

past_Events_Classname = 'calendar__row--grey'
new_day_Classname = 'calendar__row--new-day'

#if an event is not a past event, add it to the list
for i in all_events:
    if past_Events_Classname not in i['class']:
        coming_soon.append(i)

#filter out important events
for j in coming_soon:
    #print("Coming soon: ", j, "\n")
    high_imp.append(j.find("span", {"title": "Medium Impact Expected"}))

for p in high_imp:
    if p != None:
        print("High: ", p.parent.parent)
    else:
        high_imp.remove(p)
    
for m in high_imp:
    if m != None:
        single_item = m.parent.parent
        print("This was high: \n", single_item.prettify())
        imp_events.append(single_item)
    #imp_events.append(high_imp.parent())
    #print(high_imp)
    #if high_imp != None:
    #    imp_events.append(j)
    #else:
    #    pass

for k in imp_events:
    print("Important: ", k, "\n")

High:  <tr class="calendar__row calendar__row--no-grid" data-event-id="131516" data-touchable=""><td class="calendar__cell calendar__date"><!-- --></td> <td class="calendar__cell calendar__time"><!-- --></td> <td class="calendar__cell calendar__sub crunched nowrap"><div><a class="mini-sub mini-sub" href="/login" title="Subscribe to Caixin Services PMI"><span class="icon icon--mini-unsubscribed"></span></a> <!-- --> <!-- --></div></td> <td class="calendar__cell calendar__currency"><span class="">CNY</span></td> <td class="calendar__cell calendar__impact"><span class="icon icon--ff-impact-ora" title="Medium Impact Expected"></span> <div class="calendar__impact-icon calendar__impact-icon--print"><img alt="" height="12" src="https://resources.faireconomy.media/images/sprites/ff-impact-ora.png" width="14"/></div></td> <td class="calendar__cell calendar__event event" colspan="1"><div><span class="calendar__event-title">Caixin Services PMI</span></div></td> <!-- --> <td class="calendar__cell 

8. Package the details before sending to a sql database

In [85]:
#Let us list the details we want to extract from each important item:
time = ''
title = ''
expected = ''
previous = ''

#Empty list that will hold all the news items
my_posts = []

#Populate the list with the news items
for n in imp_events:
    time = n.find("td", {"class": "calendar__time"})
    if time.div is not None:
        time = type(time.div.children[0])
    else:
        time = "N/A"
    currency = n.find("td", {"class": "calendar__cell calendar__currency"}).span.text       #sorted
    title = n.find("span", {"class": "calendar__event-title"}).text                         #sorted
    previous = n.find("td", {"class": "calendar__previous"}).span.text                      #sorted
    forecast = n.find("td", {"class": "calendar__forecast"}).span.text                      #sorted
    my_posts.append({"time": time, "currency": currency, "title": title, "previous": previous, "forecast": forecast})

for mp in my_posts:
    print("\n Post: \n", mp)


TypeError: 'list_iterator' object is not subscriptable

9. Send the details to a sql database

In [ ]:
#Start a connection to the server